# Using MIP to solve an extended problem

https://developers.google.com/optimization/assignment/compare_mip_cp

## Assigment with task sizes

We could also solve this problem using the minimum cost flow approach. 

Here each tasks has a differet _size_; sizes measure the effort required to perform the task. A new constraint is that the total size of the tasks assigned to a worker must not exceed a fixed bound.




In [1]:
from ortools.linear_solver import pywraplp
import time
from __future__ import print_function


In [6]:
def main():
    """
    Solving an assignment problem with MIP
    """
    
    # Instantiate a mixed integer solver
    solver = pywraplp.Solver('SolveTransportationProblem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    
    # define the costs
    cost = [[90, 76, 75, 70, 50, 74, 12, 68],
          [35, 85, 55, 65, 48, 101, 70, 83],
          [125, 95, 90, 105, 59, 120, 36, 73],
          [45, 110, 95, 115, 104, 83, 37, 71],
          [60, 105, 80, 75, 59, 62, 93, 88],
          [45, 65, 110, 95, 47, 31, 81, 34],
          [38, 51, 107, 41, 69, 99, 115, 48],
          [47, 85, 57, 71, 92, 77, 109, 36],
          [39, 63, 97, 49, 118, 56, 92, 61],
          [47, 101, 71, 60, 88, 109, 52, 90]]
    
    task_sizes = [10, 7, 3, 12, 15, 4, 11, 5]
    
    # set the maximum total cost for any worker
    total_size_max = 15
    
    num_workers = len(cost)
    num_tasks = len(cost[0])
    
    # create the binary integer variables for the problem
    x = {}
    
    for i in range(num_workers):
        for j in range (num_tasks):
            x[i, j] = solver.IntVar(0, 1, 'x[%i,%i]' % (i, j))
            
    # constraints
    ## total size must be less than or equal to total_size_max
    for i in range(num_workers):
        solver.Add(solver.Sum([task_sizes[j] * x[i, j] for j in range(num_tasks)]) <= total_size_max)
        
    ## each task is assigned to at least 1 worker
    for j in range(num_tasks):
        solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) >= 1)    

    # create the objective function
    solver.Minimize(solver.Sum([cost[i][j] * x[i, j] for i in range(num_workers)
                                                     for j in range(num_tasks)]))
    # Invoke the solver
    sol = solver.Solve()
    
    print('Total cost = %d' % (solver.Objective().Value()))
    print()
    for i in range(num_workers):
        for j in range(num_tasks):
            if x[i, j].solution_value() > 0:
                print('worker %d assigned to task %d; cost %d.' % (i, j, cost[i][j]))
    
    print()
    print('Time = ', solver.WallTime(), " milliseconds.")
    
    

In [7]:
if __name__ == "__main__":
    start_time = time.clock()
    main()
    print()
    print("Time = ", time.clock() - start_time)

Total cost = 326

worker 0 assigned to task 6; cost 12.
worker 1 assigned to task 0; cost 35.
worker 1 assigned to task 2; cost 55.
worker 4 assigned to task 4; cost 59.
worker 5 assigned to task 5; cost 31.
worker 5 assigned to task 7; cost 34.
worker 6 assigned to task 1; cost 51.
worker 8 assigned to task 3; cost 49.

Time =  44  milliseconds.

Time =  0.03704599999999969
